# A N A
## Personal assistant

## Biblioteka narodowa

In [2]:
# ---- tkinter ----
import tkinter as tk
from tkinter import font, Label, Tk
from tkinter import PhotoImage
import cv2
from PIL import Image, ImageTk
import sounddevice as sd
import numpy as np
import threading
import wavio
# ---- open ai ---- 
import openai
import json
import winsound
from elevenlabslib import *
from pydub import AudioSegment
from pydub.playback import play
import io
from scipy.io import wavfile
import config
import struct
import wave
# ---- ElevenLabs ----
import elevenlabs
import soundfile as sf
import ctypes as ct
# ---- Google ----
from googlesearch import search
import webbrowser
import random
import os
import requests
from io import BytesIO

# ---- API ----
with open('apiKEYS.txt') as json_file:
    keys_data = json.load(json_file)
openaiKEY = keys_data["openaiKEY"]
elevenlabsKEY = keys_data["elevenlabsKEY"]
openai.api_key = openaiKEY
elevenlabs.set_api_key(elevenlabsKEY)

with open('PERSONALITIES.txt', 'r', encoding='utf-8') as file:
    system = json.load(file)
system = system["ANA"][0]
searcher = system["Searcher"]
textgen = system["TEXTGEN"]
textForGPT = system["GPT"]
imageGPT = system["GPTIMAGES"]
properties = system("PROPERTIES")

# Description
## This is ANA, personal AI assistant capable of doing unexceptional things, such as
## creating images, text files, responding with natural voice and browsing the internet.

In [ ]:
# Respect her personality and have fun talking to her

# Final Code

In [3]:
# ---- Open the video file ----
video_capture = cv2.VideoCapture("black_fast.mp4")
output_wav_path = "output.wav"
voiceID = "EXAVITQu4vr4xnSDxMaL"
messages = ['']

################################################################
# ---- Functions That Create Magic ----
################################################################
# ---- GPT4 web browser ----
def GPT_search(query):
    searcher = searcher
    response = openai.ChatCompletion.create(model="gpt-4", messages=[{"role": "assistant", "content": searcher}, 
                                                                     {"role": "system", "content": query}], max_tokens=4000, stop=None, temperature=0.5)
    query = response.choices[0].message.content
    urls = []
    for j in search(query, tld="co.in", num=8, stop=8, pause=2):
        urls.append(j)
    source_links = {}
    for link in urls:
        source = link.split("//")[1].split("/")[0]
        if source not in source_links:
            source_links[source] = []
        source_links[source].append(link)
    selected_links = []
    for source, links in source_links.items():
        selected_links.append(random.choice(links))
    selected_links = selected_links[0:4]
    for url in selected_links:
        webbrowser.open(url)
        print(url)
    
# ---- GPT4 text file generator ----
def GPTtextgenerator(gpt_input):
    text = textForGPTI
    response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "assistant", "content": gpt_input}, {"role": "system", "content": text}], max_tokens=1000, stop=None, temperature=0.5)
    response = response.choices[0].message.content

    file_name = "GPT4response.txt"
    with open(file_name, 'w') as file:
        file.write(response)
    os.system("notepad.exe " + file_name)
    
# ---- Image generator ----
def GPT_image(prompt):
    text = imageGPT
    response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "assistant", "content": prompt}, 
                                                                             {"role": "system", "content": text}], max_tokens=1000, stop=None, temperature=0.5)
    prompt = response.choices[0].message.content
    print(prompt)
    response = openai.Image.create(
        prompt = prompt, 
        n=1,
        size="1024x1024"
    )
    image_url = response['data'][0]['url']
    response = requests.get(image_url)
    image_data = BytesIO(response.content)
    image = Image.open(image_data)
    image.save("AI ig/igLotismall.png")
    image.show()
################################################################


################################################################
# ---- The rest of the code ----
################################################################

# ---- Generate window ----
root = tk.Tk()
root.title("ANA Assistant")
icon = tk.PhotoImage(file="ANAlogo.png")
root.iconphoto(False, icon)
root.configure(bg='#000000')

# ---- Calculate the desired canvas size based on video dimensions ----
frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH) - 600)
frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT) - 800)
canvas_width = frame_width + 300
canvas_height = frame_height + 300
canvas = tk.Canvas(root, width=canvas_width, height=canvas_height, bg='#000000', highlightthickness=0)
canvas.grid(row=1, column=0, columnspan=2)

# ---- Caption-related variables ----
caption_texts = ["#3282f5", "#40e3a7", "#f7a068", "#6dedea"] # colors of the dot
current_caption_index = 0

# ---- Audio-related vaiables ----
is_recording = False
audio_data = []
record_thread = None

# ---- Recording functions ----
# - Start recording
def start_recording():
    global is_recording, audio_data
    is_recording = True
    audio_data = []
    def callback(indata, frames, time, status):
        if status:
            print(status, flush=True)
        audio_data.append(indata.copy())
    with sd.InputStream(callback=callback):
        while is_recording:
            sd.sleep(0)
# - Stop recording
def stop_recording():
    global is_recording
    is_recording = False
    sd.sleep(0)
    audio_array = np.concatenate(audio_data, axis=0)
    sf.write("output.wav", audio_array, samplerate=44100)
    
# ---- Process message ----
def process_system_message(system_message):
    colon_index = system_message.find(":")
    if colon_index != -1:
        processed_message = system_message[colon_index + 1:].strip()
        return processed_message
    else:
        return system_message

# ---- AI audio transcription ----
def transcribe_audio(audio):
    global current_caption_index
    audio_file = open(audio, "rb")
    transcript = openai.Audio.transcribe("whisper-1", audio_file)
    textForGPT = transcript['text']
# ---- Here we choose which action should we use ----
    check_text = textForGPT.lower()
    function = 0
    if properties[0] in check_text:
        textForGPT = textForGPT[0]
        function = 1
    elif properties[1] in check_text:
        textForGPT = textForGPT[1]
        function = 2
    elif properties[2] in check_text:
        textForGPT = textForGPT[2]
        function = 3
    else:
        textForGPT = textForGPT
    
    ################################################################
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", 
        messages=[{"role": "assistant", "content": textForGPT}, {"role": "system", "content": system}], 
        max_tokens=120, 
        stop=None, 
        temperature=0.68)
    
    system_message = response["choices"][0].message.content
    voice = elevenlabs.Voice(
        voice_id = voiceID,
        settings = elevenlabs.VoiceSettings(
            stability = 0.37,
            similarity_boost = 0.80))
    #system_message = process_system_message(system_message)
    audio = elevenlabs.generate(
        text = system_message,
        voice = voice,
        model='eleven_multilingual_v2')
    audio = AudioSegment.from_file(io.BytesIO(audio), format="mp3")
    audio.export("output.wav", format="wav")
    current_caption_index = 3
    update_caption()
    ################################################################
    # Check what's goin' on
    ################################################################
    # function pass
    if function == 1:
        GPT_search(check_text)
    elif function == 2:
        GPTtextgenerator(check_text)
    elif function == 3:
        GPT_image(check_text)
    print(textForGPT)
        

# ---- Play AI ----
def play_audio():
    winsound.PlaySound("output.wav", winsound.SND_FILENAME)
# Thread to play audio by enter key
def enter_key_pressed(event):
    current_caption_index = 0 
    update_caption()
    audio_thread = threading.Thread(target=play_audio)
    audio_thread.start()
    
# ---- Interaction ----
def space_key_pressed(event):
    global record_thread, current_caption_index
    if not is_recording:
        record_thread = threading.Thread(target=start_recording)
        record_thread.start()
        current_caption_index = 1 
        update_caption()
    else:
        stop_recording()
        current_caption_index = 2 
        update_caption()
        audio_thread = threading.Thread(target=transcribe_audio, args=("output.wav",))
        audio_thread.start()

# ---- Change the dot color ----
def update_caption():
    caption_label.config(fg=caption_texts[current_caption_index])
    
# ---- Add key functions to the window ---- 
root.bind("<space>", space_key_pressed)
root.bind("<Return>", enter_key_pressed)

# ---- Add video ----
def update_frame():
    ret, frame = video_capture.read()
    if not ret:  # Video has ended, reset the video capture
        video_capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
        ret, frame = video_capture.read()
    if ret:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_height, frame_width, _ = frame.shape
        aspect_ratio = frame_width / frame_height
        # Ballsize changes the ballsize :)
        ballsize = canvas_height - 400
        new_width = int(ballsize * aspect_ratio)
        new_height = ballsize
        frame = cv2.resize(frame, (new_width, new_height))
        photo = ImageTk.PhotoImage(image=Image.fromarray(frame))
        x = (canvas_width - frame.shape[1]) // 2
        y = (canvas_height - frame.shape[0]) // 2 + 180
        canvas.create_image(x, y, image=photo, anchor=tk.NW)
        canvas.photo = photo
    canvas.after(10, update_frame)
update_frame()

# ---- Dark title bar ----
def dark_title_bar(window):
    window.update()
    DWMWA_USE_IMMERSIVE_DARK_MODE = 20
    set_window_attribute = ct.windll.dwmapi.DwmSetWindowAttribute
    get_parent = ct.windll.user32.GetParent
    hwnd = get_parent(window.winfo_id())
    rendering_policy = DWMWA_USE_IMMERSIVE_DARK_MODE
    value = 2
    value = ct.c_int(value)
    set_window_attribute(hwnd, rendering_policy, ct.byref(value), ct.sizeof(value))
dark_title_bar(root)

# ---- Display captions on the canvas ----
color = caption_texts[current_caption_index]
caption_label = tk.Label(root, text="\nl", bg='#000000', fg=color, font=('Wingdings', 18))
caption_label.grid(row=0, column=0, columnspan=2, sticky='n')  # Placed above the canvas


# ---- Configure row and column weights ----
root.grid_rowconfigure(0, weight=0)  # Prevent caption from expanding
root.grid_rowconfigure(1, weight=1)
root.grid_columnconfigure(0, weight=1)
root.grid_columnconfigure(1, weight=0)

# ---- Start window ----
root.mainloop()

# ---- Release the video capture and close the window on exit ----
video_capture.release()
cv2.destroyAllWindows()

################################################################

Dawno się z tobą nie słyszałem. Co tam u ciebie ciekawego?


In [5]:
# Kod autorstwa Adriana Zaręby